In [3]:
import scrapy
import requests
from scrapy.http import HtmlResponse

In [4]:
URL = 'https://www.networkworld.com/cloud-computing/'
#response = HtmlResponse(url=URL)
html = requests.get( URL ).content
sel = scrapy.Selector( text = html )

In [5]:
sel.xpath('//a[contains(@class, "card")]/@href').extract()

['https://www.networkworld.com/article/1250120/startup-oxide-computing-seeks-to-put-the-cloud-back-in-on-prem-private-clouds.html',
 'https://www.networkworld.com/article/1251939/ibm-cloud-service-aims-to-deliver-secure-multicloud-connectivity.html',
 'https://www.networkworld.com/article/1251281/eu-approves-1-3b-in-aid-for-cloud-edge-computing.html',
 'https://www.networkworld.com/article/1250044/mainframe-modernization-gets-a-boost-from-kyndryl-aws-collaboration.html',
 'https://www.networkworld.com/article/1249451/cisco-aws-further-integrate-cloud-management-capabilities.html',
 'https://www.networkworld.com/article/1247908/alibaba-scraps-plans-to-hive-off-cloud-business-amid-growing-chip-uncertainties.html',
 'https://www.networkworld.com/article/1247268/microsofts-maia-ai-azure-cobalt-chips-to-rev-up-efficiency-performance.html',
 'https://www.networkworld.com/article/1247134/cloud-management-skills-gap-drives-hybrid-cloud-adoption.html',
 'https://www.networkworld.com/article/957

In [58]:
data = {}
for a in sel.xpath("//a[@class and contains(concat(' ', normalize-space(@class), ' '), ' card ')]"):  
    date = a.xpath('div[contains (@class, "card__info--light")]/span[1]/text()').extract()[0]
    if 'dec' in date.lower():
        data[a.xpath('h4/text()').extract()[0]] = [a.xpath('@href').extract()[0], date]
data

{"Oxide puts the 'cloud' back in on-prem private clouds": ['https://www.networkworld.com/article/1250120/startup-oxide-computing-seeks-to-put-the-cloud-back-in-on-prem-private-clouds.html',
  'Dec 12, 2023'],
 'IBM cloud service aims to deliver secure, multicloud connectivity': ['https://www.networkworld.com/article/1251939/ibm-cloud-service-aims-to-deliver-secure-multicloud-connectivity.html',
  'Dec 07, 2023'],
 'EU approves $1.3B in aid for cloud, edge computing\n': ['https://www.networkworld.com/article/1251281/eu-approves-1-3b-in-aid-for-cloud-edge-computing.html',
  'Dec 05, 2023'],
 'The State of Healthcare Call Centers - Industry Report': ['https://us.resources.networkworld.com/resources/the-state-of-healthcare-call-centers-industry-report-3?utm_source=rss-feed&utm_medium=rss&utm_campaign=feed',
  '15 Dec 2023'],
 'From Cost Center to Value Generator: Transforming Healthcare Contact Centers with Salesforce & Conversational AI': ['https://us.resources.networkworld.com/resources/